# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [57]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [58]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [59]:
#lower case

spaceship.columns=spaceship.columns.str.lower()


In [60]:
#handle cabin granularity
spaceship['cabin']=spaceship['cabin'].str[0]

In [61]:
#drop name and passenger ID

spaceship=spaceship.drop(['name', 'passengerid'], axis=1)

In [62]:
#let´s normalize numerical columns

#first separate categorical from numerical columns & drop transported
cat_cols=spaceship.select_dtypes(exclude='number').columns
cat_cols=cat_cols.drop('transported')

#get numerical columns 
num_cols=spaceship.select_dtypes(include='number')
num_cols_df=pd.DataFrame(num_cols)


In [63]:
#let´s normalize numerical cols, first lets set up the normalizer parameters
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(num_cols_df)


StandardScaler()

In [64]:
#now let´s apply them
num_cols_df_scaled=scaler.transform(num_cols_df)

In [65]:
num_cols_df_scaled=pd.DataFrame(num_cols_df_scaled)

In [66]:
num_cols_df_scaled

,0,1,2,3,4,5
0,0.702095,-0.337025,-0.284274,-0.287317,-0.273736,-0.266098
1,-0.333233,-0.173528,-0.278689,-0.245971,0.209267,-0.227692
2,2.013510,-0.272527,1.934922,-0.287317,5.634034,-0.223327
3,0.287964,-0.337025,0.511931,0.326250,2.655075,-0.097634
4,-0.885407,0.117466,-0.240833,-0.037590,0.223344,-0.264352
...,...,...,...,...,...,...
8688,0.840138,-0.337025,3.947464,-0.287317,1.171754,-0.201506
8689,-0.747364,-0.337025,-0.284274,-0.287317,-0.273736,-0.266098
8690,-0.195189,-0.337025,-0.284274,2.808634,-0.272856,-0.266098
8691,0.218942,-0.337025,0.366715,-0.287317,0.036829,2.557627


In [67]:
#deal with null values 
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

num_cols_df_scaled = imputer.fit_transform(num_cols_df_scaled)

In [68]:
#Now let´s do dummies for non numerical cols
spaceship_dummies=pd.get_dummies(spaceship, columns=cat_cols)

In [74]:
#let´s deal with NaNs in the dummies df (categorical variables)

#now let´s fill the Nans
spaceship_dummies=spaceship_dummies.fillna(False)

In [76]:
spaceship_dummies=spaceship_dummies.drop(['transported'], axis=1)

In [79]:
num_cols_df_scaled=pd.DataFrame(num_cols_df_scaled)

In [ ]:
#befor TTS, set features & target
target=spaceship['transported']
features=pd.concat([spaceship_dummies, num_cols_df_scaled],axis=1)

In [88]:
#convert columns names into strings to avoid problems later
features.columns=features.columns.astype(str)

In [90]:
#let´s do train test split

X_train, X_test, y_train, y_test= train_test_split(features, target, test_size=0.20, random_state=0)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [91]:
#let´s use KNN Classifier

from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=3)

In [92]:
#fit the model to our data
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [93]:
pred=knn.predict(X_test)
pred

array([False,  True, False, ...,  True, False, False])

- Evaluate your model

In [94]:
#let´s see the score

knn.score(X_test, y_test)

0.7556066705002875

In [97]:
features.columns

Index(['homeplanet_Earth', 'homeplanet_Europa', 'homeplanet_Mars',
       'cryosleep_False', 'cryosleep_True', 'cabin_A', 'cabin_B', 'cabin_C',
       'cabin_D', 'cabin_E', 'cabin_F', 'cabin_G', 'cabin_T',
       'destination_55 Cancri e', 'destination_PSO J318.5-22',
       'destination_TRAPPIST-1e', 'vip_False', 'vip_True', '0', '1', '2', '3',
       '4', '5'],
      dtype='object')

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [99]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [98]:
grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 15, 21],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # 1 = Manhattan, 2 = Euclidean
}

- Run Grid Search

In [100]:
knn=KNeighborsClassifier()

model= RandomizedSearchCV(estimator=knn, param_distributions=grid, n_iter=50, cv=5, n_jobs=-1)

- Evaluate your model

In [101]:
model.fit(X_train, y_train)

/opt/anaconda3/lib/python3.13/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 32 is smaller than n_iter=50. Running 32 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'n_neighbors': [1, 3, 5, 7, 9, 11, 15,
                                                        21],
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']})

In [102]:
model.best_params_

{'weights': 'uniform', 'p': 2, 'n_neighbors': 21}

In [103]:
best_model = model.best_estimator_

In [104]:
pred=best_model.predict(X_test)

In [105]:
best_model.score(X_test, y_test)

0.78205865439908